In [1]:
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import MDSWriter, LocalDataset
from tqdm import tqdm
from typing import List
import torch
import json

class ListOfDict(Encoding):
    def encode(self, obj: List[dict]) -> bytes:
        # Convert the list of dictionaries to a JSON-encoded string
        json_str = json.dumps(obj)
        return json_str.encode('utf-8')

    def decode(self, data: bytes) -> List[dict]:

        # Decode the JSON-encoded string back to a list of dictionaries
        json_str = data.decode('utf-8')
        return json.loads(json_str)

# Register the custom encoding for 'list_of_dict'
_encodings['list_of_dict'] = ListOfDict

In [2]:
from glob import glob

files = glob('prepared*.jsonl')
files

['prepared-combine-ms.jsonl',
 'prepared-combine-en.jsonl',
 'prepared-llava-en.jsonl',
 'prepared-llava-ms.jsonl',
 'prepared-audio-en.jsonl',
 'prepared-malay.jsonl',
 'prepared-audio-ms.jsonl']

In [3]:
columns = {
    'conversations': 'list_of_dict',
    'filename': 'list_of_dict'
}

hashes = 'sha1', 'xxh64'

In [8]:
!rm -rf mosaic-multimodal

In [9]:
with MDSWriter(out='mosaic-multimodal', columns=columns, compression=None, hashes=hashes) as out:
    for f in files:
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                for i in range(len(l['filename'])):
                    l['filename'][i] = l['filename'][i].replace('/output-audio', '/filter-audio')
                for i in range(len(l['conversations'])):
                    l['conversations'][i]['content'] = l['conversations'][i]['content'].replace('\n<image>', ' <image>').replace('<image>\n', '<image>').replace('\n<audio>', ' <audio>').replace('<audio>\n', '<audio>').strip()
                    l['conversations'][i]['content'] = l['conversations'][i]['content'].replace('<image>', '<image> </image>').replace('<audio>', '<audio> </audio>')
                out.write(l)

37252it [00:03, 11199.87it/s]
37394it [00:03, 11806.65it/s]
156564it [00:04, 34165.90it/s]
156564it [00:04, 32687.46it/s]
44669it [00:02, 21961.96it/s]
52393it [00:02, 20767.87it/s]
44669it [00:02, 20151.87it/s]


In [10]:
l['conversations']

[{'role': 'user',
  'content': '<audio> </audio>Bagaimanakah kejelasan suara penyanyi berbanding bunyi bes dalam audio?'},
 {'role': 'assistant',
  'content': 'Kejernihan suara penyanyi lebih menyerlah berbanding bunyi bes dalam audio. Walau bagaimanapun, bunyi bass masih kuat dan berkualiti, tetapi vokalnya lebih menyerlah. Kualiti audio ini termasuk dalam kategori yang sangat baik, bukan purata atau di bawah purata. Oleh itu, walaupun saya katakan bass lemah, ia masih lebih kuat daripada pembesar suara biasa yang biasa kita miliki.'},
 {'role': 'user',
  'content': 'Bagaimanakah anda menerangkan kualiti audio jika suara penyanyi dan bunyi bes kedua-duanya jelas dan seimbang?'},
 {'role': 'assistant',
  'content': 'Jika suara penyanyi dan bunyi bes kedua-duanya jelas dan seimbang, saya akan menggambarkan kualiti audio sebagai sangat baik. Dalam kes ini, vokal mahupun bass tidak mengatasi yang lain; sebaliknya, mereka saling melengkapi dan mencipta bunyi yang kaya dan penuh. Keseimbang

In [7]:
files[0]

'prepared-combine-ms.jsonl'